# The Warehouse Location Problem
ref: https://dirkschumacher.github.io/ompr/articles/milp-problem-warehouse-location.html


**Given a set of customers and set of locations to build warehoses.
The task is to decide where to build warehouses and from what warehouses goods should be shipped to which customer.**

Two decisons that need to made at once:

**1. Where and if to build warehouses.**

**2. The assignment of customers to warehouses.**

The simple setting also implies that at least one warehouse has to be built and it's big enough to serve all customers. 


pratical example: where and if to select school bus stop, and the assignment of students to bus stops.  

# Math model

## Set and parameters
1. Set of warehouses: $ I = \{1 \ldots m\} $
2. Set of customers: $ J = \{1 \ldots n\} $

3. Transportation cost from a warehouse to a customer: $TCost_{ij}$ 
4. Fixed cost associated with each warehouse if it will be built: $FCost_{i}$

# Variables
1. wether a warehouse is selected or not: $x_{i} \in \{0,1\}$ 

2. assign customer to which warehouse: $y_{i,j} \in \{0,1\}$

# Objective
Minimize the cost of the transpotation and fixed cost 

$$\sum_i\sum_j TCost_{ij}*y_{ij} + \sum_i FCost_{i} * x_{i} $$ 

# Constraints
1. each customer can only assigned to one warehouse:
$$\sum\limits_{i=1}^{m} y_{ij} = 1,  j = 1 \ldots n  $$

2. if a warehouse is not selected $x_i=0$, the customer-warehouse relation does not exist $y_{ij} = 0$;

   if a warehouse is seleled $x_i=1$, the customer-warehouse relation may exist:$y_{ij} = 0 or 1$.

$$y_{ij} \le x_{i}, i = 1 \ldots m ,  j = 1 \ldots n  $$

## Question: 
## The above is called uncapaciated problem. What if the customers have demand and warehouse has capacity?
$$$$


**$$\sum\limits_{j}^{m}demand_{j} * y_{ij} \leq \text{Capacity}_{i} * x_{j} , i = 1 \ldots m  $$**

# Model formulaion

In [20]:
from pyomo.environ import *
import numpy as np
import pandas as pd

np.random.seed(1)
grid_size = 1000 # X,Y coordinates limit
n_customer = 100 # customers

customer_location = pd.DataFrame({
'x': np.random.randint(1, grid_size, n_customer),
'y': np.random.randint(1, grid_size, n_customer)
    },
 index = range(n_customer)
)

m_warehouse = 20  # warehouses
warehouse_location = pd.DataFrame({
    'x': np.random.randint(1, grid_size, m_warehouse),
    'y': np.random.randint(1, grid_size, m_warehouse),
},
    index = range(m_warehouse)
)


# The warehouses are also randomly placed on the grid. The fixed cost for the warehouses 
# are randomly generated as well with mean cost of 10,000.
Fcost = np.random.normal(10000, scale= 5000, size = m_warehouse)

In [21]:
# eclidian distance
def transport_cost(point1, point2):
    return np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)


Tcost = np.zeros((m_warehouse, n_customer), dtype=np.float)

for i in range(m_warehouse):
    for j in range(n_customer):
        Tcost[i,j] = transport_cost(warehouse_location.loc[i], customer_location.loc[j])
Tcost = pd.DataFrame(Tcost)        

In [22]:
type(Tcost)

pandas.core.frame.DataFrame

In [23]:
M = ConcreteModel()
# M.warehouse = RangeSet(m_warehouse)
# M.customer  = RangeSet(n_customer)
# warehouse = np.arange(m_warehouse)
# customer  = np.arange(n_customer)
warehouse = list(range(m_warehouse))
customer  = list(range(n_customer))


M.warehouse = Set(initialize = warehouse)
M.customer  = Set(initialize = customer)
# warehouse = range(m_warehouse)
# customer  = range(n_customer)

In [24]:
Tcost = Tcost.stack().to_dict()

In [25]:
## Variables
M.x = Var(M.warehouse, domain = Binary)

In [26]:
M.y = Var(M.warehouse, M.customer, domain = Binary)

In [27]:
M.Tcost = Param(M.warehouse, M.customer, initialize = Tcost)

In [28]:
Fcost1 = dict(zip(warehouse, Fcost))
M.Fcost = Param(M.warehouse, initialize = Fcost1)

In [29]:
## Objective two parts
M.obj1 = Expression(expr = sum(M.y[i,j]* Tcost[i,j]  for i in M.warehouse  for j in M.customer))



In [30]:
# M.obj2 = Expression(expr = sum(Fcost[i] * M.x[i] for i in M.warehouse))
M.obj2 = Expression(expr = summation(M.Fcost, M.x))


In [31]:
M.obj = Objective(expr= M.obj1 + M.obj2,
                  sense = minimize)

In [32]:
## constraints
# each customer can only assigned to one warehouse:
def _con_customer(M, j):
    return sum(M.y[i,j] for i in M.warehouse) == 1

M._con_customer = Constraint(M.customer, rule = _con_customer)

In [33]:
## constraints
## y[i,j] <= x[i]
def _con_relation(model, i,j ):
    return (M.y[i,j] <= M.x[i])
M._con_relation = Constraint(M.warehouse, M.customer, rule = _con_relation)

In [16]:
M.pprint()
# customer.tolist()

5 Set Declarations
    Tcost_index : Dim=0, Dimen=2, Size=20, Domain=None, Ordered=False, Bounds=None
        Virtual
    _con_relation_index : Dim=0, Dimen=2, Size=20, Domain=None, Ordered=False, Bounds=None
        Virtual
    customer : Dim=0, Dimen=1, Size=10, Domain=None, Ordered=False, Bounds=(0, 9)
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    warehouse : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=(0, 1)
        [0, 1]
    y_index : Dim=0, Dimen=2, Size=20, Domain=None, Ordered=False, Bounds=None
        Virtual

2 Param Declarations
    Fcost : Size=2, Index=warehouse, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 : 11595.195480285493
          1 :  8753.148122612949
    Tcost : Size=20, Index=Tcost_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (0, 0) :  799.9931249704588
        (0, 1) :  785.2299026399847
        (0, 2) :  656.5668282817827
        (0, 3) : 377.73668077114246
        (0, 4) :  538.88217636

In [34]:
solver = SolverFactory('cbc')
solver.solve(M,tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Sep 10 2019 

command line - /home/kon6750/miniconda3/bin/cbc -printingOptions all -import /tmp/tmphhe6d8r8.pyomo.lp -stat=1 -solve -solu /tmp/tmphhe6d8r8.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2100 (-1) rows, 2020 (-1) columns and 6000 (-1) elements
Statistics for presolved model
Original problem has 2020 integers (2020 of which binary)
Presolved problem has 2020 integers (2020 of which binary)
==== 0 zero objective 2006 different
==== absolute objective values 2006 different
==== for integers 0 zero objective 2006 different
==== for integers absolute objective values 2006 different
===== end objective counts


Problem has 2100 rows, 2020 columns (2020 with objective) and 6000 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 2020 of type 0.0-

{'Problem': [{'Name': 'unknown', 'Lower bound': 46289.42002998, 'Upper bound': 46289.42002998, 'Number of objectives': 1, 'Number of constraints': 2100, 'Number of variables': 2020, 'Number of binary variables': 2020, 'Number of integer variables': 2020, 'Number of nonzeros': 2020, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.31, 'Wallclock time': 0.27, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.3301665782928467}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [35]:
for i in warehouse:
    print('warehouse',M.x[i]())

warehouse 1.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 0.0
warehouse 1.0


In [42]:
for i in warehouse:
    for j in customer:
        print('warehouse %d customer %d'%(i,j), M.y[i,j]()) 

warehouse 0 customer 0 0.0
warehouse 0 customer 1 0.0
warehouse 0 customer 2 1.0
warehouse 0 customer 3 1.0
warehouse 0 customer 4 1.0
warehouse 0 customer 5 1.0
warehouse 0 customer 6 1.0
warehouse 0 customer 7 0.0
warehouse 0 customer 8 1.0
warehouse 0 customer 9 1.0
warehouse 0 customer 10 0.0
warehouse 0 customer 11 0.0
warehouse 0 customer 12 1.0
warehouse 0 customer 13 0.0
warehouse 0 customer 14 1.0
warehouse 0 customer 15 1.0
warehouse 0 customer 16 1.0
warehouse 0 customer 17 0.0
warehouse 0 customer 18 1.0
warehouse 0 customer 19 0.0
warehouse 0 customer 20 0.0
warehouse 0 customer 21 1.0
warehouse 0 customer 22 1.0
warehouse 0 customer 23 1.0
warehouse 0 customer 24 1.0
warehouse 0 customer 25 0.0
warehouse 0 customer 26 1.0
warehouse 0 customer 27 1.0
warehouse 0 customer 28 1.0
warehouse 0 customer 29 1.0
warehouse 0 customer 30 1.0
warehouse 0 customer 31 1.0
warehouse 0 customer 32 1.0
warehouse 0 customer 33 1.0
warehouse 0 customer 34 1.0
warehouse 0 customer 35 0.0
wa

warehouse 6 customer 21 0.0
warehouse 6 customer 22 0.0
warehouse 6 customer 23 0.0
warehouse 6 customer 24 0.0
warehouse 6 customer 25 0.0
warehouse 6 customer 26 0.0
warehouse 6 customer 27 0.0
warehouse 6 customer 28 0.0
warehouse 6 customer 29 0.0
warehouse 6 customer 30 0.0
warehouse 6 customer 31 0.0
warehouse 6 customer 32 0.0
warehouse 6 customer 33 0.0
warehouse 6 customer 34 0.0
warehouse 6 customer 35 0.0
warehouse 6 customer 36 0.0
warehouse 6 customer 37 0.0
warehouse 6 customer 38 0.0
warehouse 6 customer 39 0.0
warehouse 6 customer 40 0.0
warehouse 6 customer 41 0.0
warehouse 6 customer 42 0.0
warehouse 6 customer 43 0.0
warehouse 6 customer 44 0.0
warehouse 6 customer 45 0.0
warehouse 6 customer 46 0.0
warehouse 6 customer 47 0.0
warehouse 6 customer 48 0.0
warehouse 6 customer 49 0.0
warehouse 6 customer 50 0.0
warehouse 6 customer 51 0.0
warehouse 6 customer 52 0.0
warehouse 6 customer 53 0.0
warehouse 6 customer 54 0.0
warehouse 6 customer 55 0.0
warehouse 6 customer

warehouse 10 customer 27 0.0
warehouse 10 customer 28 0.0
warehouse 10 customer 29 0.0
warehouse 10 customer 30 0.0
warehouse 10 customer 31 0.0
warehouse 10 customer 32 0.0
warehouse 10 customer 33 0.0
warehouse 10 customer 34 0.0
warehouse 10 customer 35 0.0
warehouse 10 customer 36 0.0
warehouse 10 customer 37 0.0
warehouse 10 customer 38 0.0
warehouse 10 customer 39 0.0
warehouse 10 customer 40 0.0
warehouse 10 customer 41 0.0
warehouse 10 customer 42 0.0
warehouse 10 customer 43 0.0
warehouse 10 customer 44 0.0
warehouse 10 customer 45 0.0
warehouse 10 customer 46 0.0
warehouse 10 customer 47 0.0
warehouse 10 customer 48 0.0
warehouse 10 customer 49 0.0
warehouse 10 customer 50 0.0
warehouse 10 customer 51 0.0
warehouse 10 customer 52 0.0
warehouse 10 customer 53 0.0
warehouse 10 customer 54 0.0
warehouse 10 customer 55 0.0
warehouse 10 customer 56 0.0
warehouse 10 customer 57 0.0
warehouse 10 customer 58 0.0
warehouse 10 customer 59 0.0
warehouse 10 customer 60 0.0
warehouse 10 c

warehouse 17 customer 33 0.0
warehouse 17 customer 34 0.0
warehouse 17 customer 35 0.0
warehouse 17 customer 36 0.0
warehouse 17 customer 37 0.0
warehouse 17 customer 38 0.0
warehouse 17 customer 39 0.0
warehouse 17 customer 40 0.0
warehouse 17 customer 41 0.0
warehouse 17 customer 42 0.0
warehouse 17 customer 43 0.0
warehouse 17 customer 44 0.0
warehouse 17 customer 45 0.0
warehouse 17 customer 46 0.0
warehouse 17 customer 47 0.0
warehouse 17 customer 48 0.0
warehouse 17 customer 49 0.0
warehouse 17 customer 50 0.0
warehouse 17 customer 51 0.0
warehouse 17 customer 52 0.0
warehouse 17 customer 53 0.0
warehouse 17 customer 54 0.0
warehouse 17 customer 55 0.0
warehouse 17 customer 56 0.0
warehouse 17 customer 57 0.0
warehouse 17 customer 58 0.0
warehouse 17 customer 59 0.0
warehouse 17 customer 60 0.0
warehouse 17 customer 61 0.0
warehouse 17 customer 62 0.0
warehouse 17 customer 63 0.0
warehouse 17 customer 64 0.0
warehouse 17 customer 65 0.0
warehouse 17 customer 66 0.0
warehouse 17 c

<img src="solution.png">
